# Triangle Counting Exercises

Uses the undirected Zachary Karate Club graph (insert reference) which has 45 triangles.

In [ ]:
import pygraphblas as grb
from pygraphblas.gviz import draw, draw_graph_op as draw_op
from pygraphblas.gviz import draw_cy

In [ ]:
# Build the Karate Club Graph
NUM_NODES = 34

iA = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      1,1,1,1,1,1,1,1,1,
      2,2,2,2,2,2,2,2,2,2,
      3,3,3,3,3,3,
      4,4,4,
      5,5,5,5,
      6,6,6,6,
      7,7,7,7,
      8,8,8,8,8,
      9,9,
      10,10,10,
      11,
      12,12,
      13,13,13,13,13,
      14,14,
      15,15,
      16,16,
      17,17,
      18,18,
      19,19,19,
      20,20,
      21,21,
      22,22,
      23,23,23,23,23,
      24,24,24,
      25,25,25,
      26,26,
      27,27,27,27,
      28,28,28,
      29,29,29,29,
      30,30,30,30,
      31,31,31,31,31,31,
      32,32,32,32,32,32,32,32,32,32,32,32,
      33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33]

jA = [1,2,3,4,5,6,7,8,10,11,12,13,17,19,21,31,
      0,2,3,7,13,17,19,21,30,
      0,1,3,7,8,9,13,27,28,32,
      0,1,2,7,12,13,
      0,6,10,
      0,6,10,16,
      0,4,5,16,
      0,1,2,3,
      0,2,30,32,33,
      2,33,
      0,4,5,
      0,
      0,3,
      0,1,2,3,33,
      32,33,
      32,33,
      5,6,
      0,1,
      32,33,
      0,1,33,
      32,33,
      0,1,
      32,33,
      25,27,29,32,33,
      25,27,31,
      23,24,31,
      29,33,
      2,23,24,33,
      2,31,33,
      23,26,32,33,
      1,8,32,33,
      0,24,25,28,32,33,
      2,8,14,15,18,20,22,23,29,30,31,33,
      8,9,13,14,15,18,19,20,22,23,26,27,28,29,30,31,32]
        
def BuildMatrix():
    graph = grb.Matrix.from_lists(iA, jA, [True]*len(iA), nrows=NUM_NODES, ncols=NUM_NODES, typ=grb.types.BOOL)

    print(graph)
    #print(graph.to_string(width = 4, empty_char='   -'))
    #print(list(graph))
    return graph
    
style=[{'selector': 'node',
               'style': {'background-color': 'blue',
                         'label': 'data(id)',
                         'width': 2,
                         'height': 2,
                         'shape': 'circle',
                         'color': '#000000',
                         'font-weight': 400,
                         'text-halign': 'right', 
                         'text-valign': 'bottom',
                         'font-size': 4}},
              {'selector': 'edge',
               'style': {'width': 0.2,
                         'opacity': 1,
                         #'label': 'data(id)',
                         'line-color': 'green',
                         'font-size': 4}}]

G = BuildMatrix()
draw_cy(G, visual_style=style)
#draw(G)

In [ ]:
# Exercise: num_tri = 1/6 * A .* (A +.* A)
# Counts each triangle six times: starts at each of the three vertices and goes in both directions

def triangleCount_v1(graph):
    C = graph.mxm(graph, semiring=grb.types.UINT64.PLUS_TIMES, mask=graph)
    #print(C)
    count = C.reduce_int(grb.types.UINT64.PLUS_MONOID)
    return count/6

num_triangles = triangleCount_v1(G)
print("Number of triangles (v1):", num_triangles)

In [ ]:
# Exercise: Extract the lower triangular portion and num_tri = L .* (L +.* L') = L .* (L +.* L)

def extractLower_v2a(graph):
    i,j,v = graph.to_lists()
    li = []
    lj = []
    for row,col in zip(i, j):
        if col < row:
            li.append(row)
            lj.append(col)
    # Note that you must specify nrows, ncols or it will drop at least the last column
    L = grb.Matrix.from_lists(li, lj, [True]*len(li), nrows=NUM_NODES, ncols=NUM_NODES, typ=grb.types.BOOL)
    #print("v1 nvals:", L.nvals, L.nrows, L.ncols)
    return L

def triangleCount_v2a(graph):
    L = extractLower_v2a(graph)
    C = L.mxm(L, semiring=grb.types.UINT64.PLUS_TIMES, mask=L)
    count = C.reduce_int(grb.types.UINT64.PLUS_MONOID)
    return count
    
print("Number of triangles (v2a):", triangleCount_v2a(G))


In [ ]:
# A simpler way to extract lower triangular portion Matrix.tril()
def extractLower_v2b(graph):
    # tril ensures the output matrix dimension is the same as the input
    L = graph.tril(-1)  # -1 -> don't include the main diagonal
    #print("v2 nvals:", L.nvals, L.nrows, L.ncols)
    return L

def triangleCount_v2b(graph):
    L = extractLower_v2b(graph)
    C = L.mxm(L, semiring=grb.types.UINT64.PLUS_TIMES, mask=L)
    count = C.reduce_int(grb.types.UINT64.PLUS_MONOID)
    return count
    
print("Number of triangles (v2b):", triangleCount_v2b(G))


In [ ]:
# Exercise: Performance: Sort by degree descending, then #tris = L .* (L +.* L)

def sortDescending_extractLower(graph):
    degrees = grb.Vector.sparse(grb.types.UINT64, NUM_NODES) 
    graph.reduce_vector(grb.types.UINT64.PLUS_MONOID, out=degrees)
    idx,val = degrees.to_lists()
    tuple_list = list(zip(idx,val))
    print("(vertex,degree):", tuple_list)
    
    sorted_degrees = sorted(tuple_list, key=lambda x: x[1], reverse=True)
    print("sorted(vertex,degree):", sorted_degrees)
    
    permuted_ids, sdeg = zip(*sorted_degrees)
    print("sorted vertex list:", list(permuted_ids))
    
    Gperm = graph.extract_matrix(list(permuted_ids), list(permuted_ids))
    print("Permuted adjacency matrix:", Gperm)
    
    L = Gperm.tril(-1)
    return L

def triangleCount_v3(graph):
    L = sortDescending_extractLower(graph)
    C = L.mxm(L, semiring=grb.types.UINT64.PLUS_TIMES, mask=L)
    count = C.reduce_int(grb.types.UINT64.PLUS_MONOID)
    return count
    
print("Number of triangles (v3):", triangleCount_v3(G))